# Project 7

## Sys Ops

In [1]:
cd ..

/home/alex/tmp/P7-ML-Rudy


/home/alex/tmp/P7-ML-Rudy/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
pwd

'/home/alex/tmp/P7-ML-Rudy'

## Imports 

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import missingno as msno

from sklearn.feature_selection import *
from sklearn.feature_extraction import *
from sklearn.decomposition import *
from sklearn.model_selection import *
from sklearn.impute import *
from sklearn.preprocessing import *
from sklearn.pipeline import *
from sklearn.covariance import *
from sklearn.metrics import *
from sklearn.compose import *

from sklearn.dummy import *
from sklearn.linear_model import *
from sklearn.neighbors import *
from sklearn.svm import *
from sklearn.ensemble import *
from sklearn.cluster import *


from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [4]:
# DO NOT REMOVE THIS LINE

## Data

In [24]:
fn = "df.csv"
df = pd.read_csv("./data/preprocessed/" + fn)

In [25]:
df.shape

(307454, 20)

In [26]:
df = df.sample(frac=0.10)
df

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,EXT_SOURCE_2,EXT_SOURCE_3,nan_EXT_SOURCE_1
217722,0,Cash loans,2,0,1,0,211500.0,826398.0,29682.0,697500.0,Unaccompanied,Pensioner,Secondary / secondary special,Separated,House / apartment,61.931507,NaN,0.459369,0.486653,0
232669,0,Cash loans,1,1,1,0,180000.0,855000.0,24997.5,855000.0,Family,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,49.775342,1.865753,0.582113,0.499272,1
109848,1,Cash loans,1,0,0,2,180000.0,531265.5,25969.5,373500.0,Unaccompanied,Working,Secondary / secondary special,Married,Rented apartment,39.780822,1.216438,0.368846,0.531686,1
237559,0,Cash loans,2,0,0,0,90000.0,110331.0,11713.5,103500.0,Family,Pensioner,Secondary / secondary special,Married,House / apartment,58.172603,NaN,0.421608,0.651260,1
28771,0,Cash loans,2,0,1,0,67500.0,307557.0,14926.5,265500.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,51.005479,9.649315,0.297184,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71626,0,Cash loans,2,0,1,0,157500.0,1350000.0,39604.5,1350000.0,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,43.764384,5.846575,0.608615,NaN,1
240136,0,Revolving loans,1,0,1,0,81000.0,180000.0,9000.0,180000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,58.230137,36.441096,0.597875,0.584990,0
169126,0,Cash loans,2,1,1,0,157500.0,450000.0,13644.0,450000.0,Unaccompanied,Pensioner,Higher education,Married,House / apartment,59.616438,NaN,0.752261,0.401407,0
5350,0,Cash loans,2,0,1,0,128250.0,792000.0,23287.5,792000.0,Unaccompanied,State servant,Secondary / secondary special,Married,House / apartment,46.208219,20.989041,0.344191,0.403142,0


In [27]:
X = df.drop(columns=["TARGET"])
y  = df["TARGET"]

In [28]:
y.value_counts(normalize=True).round(2)

TARGET
0    0.92
1    0.08
Name: proportion, dtype: float64

In [29]:
def resultize(grid, verbose=2):
    """make results based on gridsearchcv"""

    # make df
    results = pd.DataFrame(grid.cv_results_)

    # clean columns
    cols = [i for i in results.columns if not i.startswith("split")]
    results = results.loc[:, cols]

    if verbose == 2:
        print("coucou")

    return results.round(2)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [31]:
y_train.value_counts(normalize=True)

TARGET
0    0.919959
1    0.080041
Name: proportion, dtype: float64

In [32]:
y_test.value_counts(normalize=True)

TARGET
0    0.921939
1    0.078061
Name: proportion, dtype: float64

## Modelisation

### Refactoring avec utilisation de Pipeline 

### Création du ColumnTransformer

In [38]:
numerical_features = [
    "CODE_GENDER",
    "FLAG_OWN_CAR",
    "FLAG_OWN_REALTY",
    "CNT_CHILDREN",
    "AMT_INCOME_TOTAL",
    "AMT_CREDIT",
    "AMT_ANNUITY",
    "AMT_GOODS_PRICE",
    "DAYS_BIRTH",
    "DAYS_EMPLOYED",
    # "EXT_SOURCE_1",
    "EXT_SOURCE_2",
    "EXT_SOURCE_3",
]

categorical_features = [
    "NAME_CONTRACT_TYPE",
    "NAME_TYPE_SUITE",
    "NAME_INCOME_TYPE",
    "NAME_EDUCATION_TYPE",
    "NAME_FAMILY_STATUS",
    "NAME_HOUSING_TYPE",
    # "OCCUPATION_TYPE",
    # "ORGANIZATION_TYPE",
]

In [39]:
# Création de 2 Pipelines

numerical_pipeline = make_pipeline(KNNImputer(), StandardScaler())

categorical_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"), OneHotEncoder()
)

In [40]:
# Création du ColumnTransformer
# numerical_pipeline est appliquée sur les numerical_features
# categorical_pipeline est appliquée sur les categorical_features

ColumnTransformer = make_column_transformer(
    (numerical_pipeline, numerical_features),
    (categorical_pipeline, categorical_features),
)
ColumnTransformer

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('knnimputer', KNNImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['CODE_GENDER', 'FLAG_OWN_CAR',
                                  'FLAG_OWN_REALTY', 'CNT_CHILDREN',
                                  'AMT_INCOME_TOTAL', 'AMT_CREDIT',
                                  'AMT_ANNUITY', 'AMT_GOODS_PRICE',
                                  'DAYS_BIRTH', 'DAYS_EMPLOYED', 'EXT_SOURCE_2',
                                  'EXT_SOURCE_3']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 ['NAME_CONTRACT_TYPE', 'NAME_TYPE_SUITE',
                                  'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
                                  'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE'])])

In [46]:


pipe = Pipeline(
    [
        ("columntransformer", ColumnTransformer),
        #  ("inputer", KNNImputer()),
        #  ("scaler", StandardScaler()),
        ('under_sampler', RandomUnderSampler()),
        ("estimator", DummyClassifier()),
    ]
)

In [47]:
param_grid = {
    "estimator": [
        DummyClassifier(),
        LogisticRegression(),
        RandomForestClassifier(),
    ],
}

In [48]:
grid2 = GridSearchCV(
    pipe,
    param_grid=param_grid,
    cv=5,
    verbose=2,
    return_train_score=True,
    n_jobs=-1,
    error_score='raise'
)
grid2

GridSearchCV(cv=5, error_score='raise',
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('knnimputer',
                                                                                          KNNImputer()),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         ['CODE_GENDER',
                                                                          'FLAG_OWN_CAR',
                                                                          'FLAG_OWN_REALTY',
                                                                          'CNT_CHILDREN',
                                                                          'AMT_INCOME_TOTAL',
                                                                          'AMT_CREDIT',
                                                                          'AMT_ANNUITY',
                                                                          'AMT_GOODS_PRICE',
                                                                          'DAYS_BIRT...
                                                                                          OneHotEncoder())]),
                                                                         ['NAME_CONTRACT_TYPE',
                                                                          'NAME_TYPE_SUITE',
                                                                          'NAME_INCOME_TYPE',
                                                                          'NAME_EDUCATION_TYPE',
                                                                          'NAME_FAMILY_STATUS',
                                                                          'NAME_HOUSING_TYPE'])])),
                                       ('under_sampler', RandomUnderSampler()),
                                       ('estimator', DummyClassifier())]),
             n_jobs=-1,
             param_grid={'estimator': [DummyClassifier(), LogisticRegression(),
                                       RandomForestClassifier()]},
             return_train_score=True, verbose=2)

In [49]:
grid2.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


In [45]:
resultize(grid2)

AttributeError: 'GridSearchCV' object has no attribute 'cv_results_'

In [26]:
grid2.cv_results_

AttributeError: 'GridSearchCV' object has no attribute 'cv_results_'